<h1>Milestone 2: Network latencies</h1>
<ol>
    <h2><li>Latencies in different scenarios:</li></h2>
    <ol>
        <h3><li>Localhost</li></h3>
        <h3><li>Localhost with simulated latencies</li></h3>
        <h3><li>Localhost with simulated loss</li></h3>
        <h3><li>Internet</li></h3>
        <h3><li>Quality of Service</li></h3>
    </ol>
    <h2><li>Controlling the buffering time to hide the jitter</li></h2>
</ol>

--------------------------------------------------------------------------------------------------------------------

<h2>1. Latencies in different scenarios</h2>
<p>In this section, the latencies distribution will be characterized in different scenarios (localhost and Internet).</p>

<p>The steps to be followed are:</p>

<ol>
    <li>ping localhost -c 100 -s 4096 > ping.txt</li>
    <li>grep from < ping.txt | cut -f 4 -d "=" | cut -f 1 -d " " | awk '{print $1/2}' > localhost_latencies.txt</li>
    <li>Find the histogram:</li>

    cat << EOF | python - 

    import numpy as np 

    from scipy import stats 

    latencies = np.loadtxt("localhost_latencies.txt") 

    average_latency = np.average(latencies) 

    print("\naverage latency =", average_latency) 

    max_latency = np.max(latencies) 

    min_latency = np.min(latencies) 
    
    maximum_absolute_deviation = max(max_latency - average_latency, average_latency - min_latency) 
  
    print("maximum absolute deviation (jitter) =", maximum_absolute_deviation)   
    
    correlation_coefficient = stats.pearsonr(latencies, np.roll(latencies, 1))[0]  
    
    print("Pearson correlation coefficient =", correlation_coefficient)  
    
    if correlation_coefficient < 0: 
    
        print("Correlation coefficient < 0: use 0 (no correlation between RTT samples) in your experiments") 
         
    histogram = np.histogram(latencies) 
    
    np.savetxt("localhost_histogram.txt", histogram[0]) 
    
    EOF
</ol>

4. gnuplot (plot "localhost_histogram.txt" with histogram)</li>

<h3>1.1. Localhost</h3>
<p>The ping command will be headed to localhost, so the latency will be the minimal and there will not be any loss.</p>

<p>We can now see that the histogram has the shape of a normal distribution (as it can be seen in the following figure, which represents an example of a normal distribution).</p>

<img width="500px" alt="Distribución Normal" src="https://conceptosclaros.com/wp-content/uploads/2016/10/histograma10000Obs.png">

--------------------------------------------------------------------------------------------------------------------

<h3>1.2. Localhost with simulated latencies</h3>
<p>In this section, we will simulate latencies to represent a similar scenario to the real one. This can be done using the following command:</p>

$sudo tc qdisc add dev lo root netem delay 100ms 10ms 25% distribution normal
<p>This command will simulate a 100ms latency which will follow a normal distribution. Later on, we followed the steps determined before in order to generate the histogram, which is shown below. In fact, we can see that the histogram follows a normal distribution.</p>

--------------------------------------------------------------------------------------------------------------------

<h3>1.3. Localhost with simulated loss</h3>
<p>We will simulate a bad Internet connection by making our device lose some packets. It will be done using the following command:</p>

$sudo tc qdisc add dev lo root netem loss 10%
<p>This command will stablish a rule to simulate a loss of a 10% of the packets. Again, the histogram shape follows a normal distribution.</p>

<h3>1.4. Internet</h3>
<p>Lastly, we did this in a real environment (Internet). To do this, we headed the ping command to a partner's router. See what happened:</p>

<p>We can see that now, the distribution followed is not a normal distribution, but an exponential distribution.</p>

<img width="500px" alt="Distribución Exponencial" src="https://i.stack.imgur.com/vC1fT.gif">

<h3>Quality of Service</h3>
<p>We run the InterCom project as it was told. Obviously, in a localhost the quality of service was perfect, as we added the delay or the loss of packets, we could see that the quality was going down. In the case of a packet loss, we could hear it as a silence or</p>